# Astropy: Tables


<section class="objectives panel panel-warning">
<div class="panel-heading">
<h2><span class="fa fa-certificate"></span> Objectives</h2>
</div>


<div class="panel-body">

<ul>
<li>Create tables</li>
<li>Access data in tables</li>
<li>Combining tables</li>
<li>Aggregation</li>
<li>Masking</li>
<li>Reading/writing</li>
</ul>

</div>

</section>


## Documentation

For more information about the features presented below, you can read the
[astropy.table](http://docs.astropy.org/en/stable/table/index.html) docs.

## Creating tables

In [2]:
from astropy.table import Table
import numpy as np

t1=Table()
t1['name']=['source1','source2','source3']
t1['flux']=[1.2,2.2,3.1]

In [3]:
t1

name,flux
str7,float64
source1,1.2
source2,2.2
source3,3.1


In [5]:
t1['size']=[1,2,5]
t1

name,flux,size
str7,float64,int64
source1,1.2,1
source2,2.2,2
source3,3.1,5


In [6]:
t1['size']

1
2
5


In [7]:
np.array(t1['size'])

array([1, 2, 5])

In [8]:
t1['size'][0]

1

In [9]:
t1[0]

name,flux,size
str7,float64,int64
source1,1.2,1


In [10]:
t1[0]['size']

1

## Units in tables

In [12]:
t1['size'].unit = 'cm'
t1

name,flux,size
,,cm
str7,float64,int64
source1,1.2,1
source2,2.2,2
source3,3.1,5


Some unitful operations will then work:

In [13]:
t1['size'].to('m')

<Quantity [0.01, 0.02, 0.05] m>

However, you may run into unexpected behavior, so if you are planning on using table columns as Quantities, we recommend that you use the ``QTable`` class:

In [14]:
type(t1['size'])

astropy.table.column.Column

In [31]:
from astropy.table import QTable
qt1=QTable(t1)


In [18]:
type(qt1['size'])

astropy.units.quantity.Quantity


<section class="challenge panel panel-success">
<div class="panel-heading">
<h2><span class="fa fa-pencil"></span> Challenge</h2>
</div>


<div class="panel-body">

<ol>
<li>Make a table that contains three columns: <code>spectral type</code>, <code>temperature</code>, and <code>radius</code>, and incude 5 rows with fake data (or real data if you like, for example from <a href="http://www.atlasoftheuniverse.com/startype.html">here</a>). Try including units on the columns that can have them.</li>
<li>Find the mean temperature and the maximum radius</li>
<li>Try and find out how to add and remove rows</li>
<li>Add a new column which gives the luminosity (using $L=4\pi R^2 \sigma T^4$)</li>
</ol>

</div>

</section>


In [69]:
import numpy as np
import astropy.units as u
from astropy.table import Table
from astropy.table import QTable

#creating a QTable from a normal table

t2=Table()

t2['spectral type']=['O5','B3','A3','G5','K2','B2']
t2['temperature']=[1000000,200000,3000000,400000,500000,900000]*u.K
t2['radius']=[1.5,3,4,5,6,7]*u.R_sun



qt2=QTable(t2)
qt2

#calculating the mean temperature and max radius

# temp=np.array(qt2['temperature'])
# mean_temp=sum(temp)/len(temp)
# print(mean_temp*u.K)

mean_temp=np.mean(qt2['temperature'])
print(mean_temp)

# rad=np.array(qt2['radius'])
# max_rad=max(rad*u)

max_rad=np.max(qt2['radius'])
print(max_rad)

#Adding row with luminosity

qt2.add_row({'spectral type': 'K5','temperature':4300*u.K,'radius':0.72*u.R_sun})

#to add a row or remove rom in a specific position       qt2.insert_row()  qt2.remove_row()

#To add a column with the luminosity, multiplying the values of the other columns

from astropy.constants import sigma_sb

qt2['Luminosity']=(4*np.pi*qt2['radius']**2*sigma_sb*qt2['temperature']**4).to(u.L_sun)
qt2

1000000.0 K
7.0 solRad


spectral type,temperature,radius,Luminosity
,K,solRad,solLum
str2,float64,float64,float64
O5,1000000.0,1.5,2027102121.4181356
B3,200000.0,3.0,12973453.577076068
A3,3000000.0,4.0,1167610821936.8462
G5,400000.0,5.0,576597936.7589364
K2,500000.0,6.0,2027102121.4181356
B2,900000.0,7.0,28964045951.670887
K5,4300.0,0.72,0.15967316182594046


## Iterating over tables

It is possible to iterate over rows or over columns. To iterate over rows, simply iterate over the table itself:

In [70]:
for row in t1:
    print(row)
    

  name  flux size
              cm 
------- ---- ----
source1  1.2    1
  name  flux size
              cm 
------- ---- ----
source2  2.2    2
  name  flux size
              cm 
------- ---- ----
source3  3.1    5


Rows can act like dictionaries, so you can access specific columns from a row:

In [72]:
for row in t1:
    print(row['name'])
    

source1
source2
source3


Iterating over columns is also easy:

In [76]:
for colname in t1.colnames:
    print(t1[colname][0])

source1
1.2
1


Accessing specific rows from a column object can also be done with the item notation:

## Joining tables

In [77]:
from astropy.table import join

In [98]:
t2= Table()
t2['name']= ['source1','source 3']
t2['flux2']= [1.,9.]

In [99]:
t1

name,flux,size
,,cm
str7,float64,int64
source1,1.2,1
source2,2.2,2
source3,3.1,5


In [100]:
t2

name,flux2
str8,float64
source1,1.0
source 3,9.0


In [101]:
t3=join(t1,t2,join_type='inner')

In [102]:
t3

name,flux,size,flux2
,,cm,
str8,float64,int64,float64
source1,1.2,1,1.0


## Masked tables

In [104]:
t3['flux2'].mask

AttributeError: 'Column' object has no attribute 'mask'

## Slicing

Tables can be sliced like Numpy arrays:

In [108]:
obs = Table()
obs['name']=['M31','M32','M31','M40','M50']
obs['mag']=[4,6,8,7,6]

In [109]:
obs

name,mag
str3,int64
M31,4
M32,6
M31,8
M40,7
M50,6


In [112]:
obs[0:4]

name,mag
str3,int64
M31,4
M32,6
M31,8
M40,7


In [116]:
obs[obs['mag']>6]

name,mag
str3,int64
M31,8
M40,7


In [120]:
obs['mag','name']

mag,name
int64,str3
4,M31
6,M32
8,M31
7,M40
6,M50


In [122]:
obs

name,mag
str3,int64
M31,4
M32,6
M31,8
M40,7
M50,6



<section class="challenge panel panel-success">
<div class="panel-heading">
<h2><span class="fa fa-pencil"></span> Challenge</h2>
</div>


<div class="panel-body">

<p>Starting from the <code>obs</code> table:</p>
<ol>
<li>Make a new table that shows every other row, starting with the second row? (that is, the second, fourth, sixth, etc. rows).</li>
<li>Make a new table the only contains rows where <code>name</code> is <code>M31</code></li>
</ol>

</div>

</section>


In [130]:
obs[1::2]

name,mag
str3,int64
M32,6
M40,7


In [129]:
obs[obs['name']==['M31']]

name,mag
str3,int64
M31,4
M31,8


## Grouping and Aggregation

In [132]:
obs_by_name=obs.group_by('name')

In [136]:
for group in obs_by_name.groups:
    print(group)
    print("")

name mag
---- ---
 M31   4
 M31   8

name mag
---- ---
 M32   6

name mag
---- ---
 M40   7

name mag
---- ---
 M50   6



In [137]:
obs_by_name.groups.aggregate(np.mean)

name,mag
str3,float64
M31,6.0
M32,6.0
M40,7.0
M50,6.0


## Writing data

In [138]:
obs.write('obstable.fits')

In [145]:
obs.write('obstable.vot',format='votable')

In [146]:
!cat obstable.vot

<?xml version="1.0" encoding="utf-8"?>
<!-- Produced with astropy.io.votable version 3.0.2
     http://www.astropy.org/ -->
<VOTABLE version="1.3" xmlns="http://www.ivoa.net/xml/VOTable/v1.3" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:noNamespaceSchemaLocation="http://www.ivoa.net/xml/VOTable/v1.3">
 <RESOURCE type="results">
  <TABLE>
   <FIELD ID="name" arraysize="3" datatype="unicodeChar" name="name"/>
   <FIELD ID="mag" datatype="long" name="mag"/>
   <DATA>
    <TABLEDATA>
     <TR>
      <TD>M31</TD>
      <TD>4</TD>
     </TR>
     <TR>
      <TD>M32</TD>
      <TD>6</TD>
     </TR>
     <TR>
      <TD>M31</TD>
      <TD>8</TD>
     </TR>
     <TR>
      <TD>M40</TD>
      <TD>7</TD>
     </TR>
     <TR>
      <TD>M50</TD>
      <TD>6</TD>
     </TR>
    </TABLEDATA>
   </DATA>
  </TABLE>
 </RESOURCE>
</VOTABLE>


## Reading data

In [ ]:
t4 = Table.read()


<section class="challenge panel panel-success">
<div class="panel-heading">
<h2><span class="fa fa-pencil"></span> Challenge</h2>
</div>


<div class="panel-body">

<p>Using the <code>t4</code> table above:</p>
<ol>
<li>
<p>Make a plot that shows <code>j_m</code>-<code>h_m</code> on the x-axis, and <code>h_m</code>-<code>k_m</code> on the y-axis</p>
</li>
<li>
<p>Make a new table that contains the subset of rows where the <code>j_snr</code>, <code>h_snr</code>, and <code>k_snr</code> columns, which give the signal-to-noise-ratio in the J, H, and K band, are greater than 10, and try and show these points in red in the plot you just made.</p>
</li>
<li>
<p>Make a new table (based on the full table) that contains only the RA, Dec, and the <code>j_m</code>, <code>h_m</code> and <code>k_m</code> columns, then try and write out this catalog into a format that you can read into another software package. For example, try and write out the catalog into CSV format, then read it into a spreadsheet software package (e.g. Excel, Google Docs, Numbers, OpenOffice). You may run into an issue at this point - if so, take a look at https://github.com/astropy/astropy/issues/7357 to see how to fix it.</p>
</li>
</ol>

</div>

</section>
